In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q "/content/drive/MyDrive/2.2.Silent Speech/5-9_silent.zip" -d "/content/drive/MyDrive/2.2.Silent Speech"

In [ ]:
import os
import json
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Set the working directory
os.chdir('/content/drive/MyDrive/2.2.Silent Speech/5-9_silent')

# Load the data
X, y = [], []
for file in os.listdir():
    if file.endswith('_emg.npy'):
        # Extract the label from the corresponding json file
        with open(file.replace('_emg.npy', '_info.json'), 'r') as f:
            info = json.load(f)
            sentence_index = info['sentence_index']
            # Exclude samples with sentence_index -1
            if sentence_index != -1:
                emg = np.load(file)
                X.append(emg)
                y.append(sentence_index)

# Pad sequences to the same length
X = pad_sequences(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = Sequential()
model.add(LSTM(100, input_shape=(None, 8)))

# Adjust the number of units in the output layer to match the number of classes in your labels
num_classes = max(y) + 1  # Assuming labels start from 0
model.add(Dense(num_classes, activation='softmax'))  # Use softmax for multi-class classification

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
# Train the model and save the history
history = model.fit(np.array(X_train), np.array(y_train), epochs=100, validation_data=(np.array(X_test), np.array(y_test)))

Epoch 1/100
6/6 [==============================] - 110s 18s/step - loss: 7.0522 - accuracy: 0.0000e+00 - val_loss: 7.0329 - val_accuracy: 0.0000e+00
Epoch 2/100
6/6 [==============================] - 116s 20s/step - loss: 6.8235 - accuracy: 0.0058 - val_loss: 7.0812 - val_accuracy: 0.0000e+00
Epoch 3/100
6/6 [==============================] - 116s 20s/step - loss: 6.6102 - accuracy: 0.0058 - val_loss: 7.1576 - val_accuracy: 0.0000e+00
Epoch 4/100
6/6 [==============================] - 116s 20s/step - loss: 6.3467 - accuracy: 0.0058 - val_loss: 7.3062 - val_accuracy: 0.0000e+00
Epoch 5/100
6/6 [==============================] - 116s 19s/step - loss: 6.0944 - accuracy: 0.0116 - val_loss: 7.4457 - val_accuracy: 0.0000e+00
Epoch 6/100
6/6 [==============================] - 116s 20s/step - loss: 5.8977 - accuracy: 0.0116 - val_loss: 7.6079 - val_accuracy: 0.0000e+00
Epoch 7/100
6/6 [==============================] - 122s 21s/step - loss: 5.7367 - accuracy: 0.0116 - val_loss: 7.7862 - val_ac

In [ ]:
# Import matplotlib
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)

print(f'Test loss: {loss:.3f}')
print(f'Test accuracy: {accuracy:.3f}')


In [ ]:
# Make a prediction
prediction = model.predict(np.array([X_test[0]]))

# The prediction is an array of probabilities for each class, so we take the class with the highest probability
predicted_class = np.argmax(prediction)

print(f'Predicted class: {predicted_class}')


In [ ]:
# Save the model
model.save('/content/sample_data/my_model.h5')


In [1]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('/content/sample_data/my_model.h5')


In [2]:
import tensorflow as tf

# Create a converter object from the Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)

# Set the supported operations to include TensorFlow operations
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # TFLite default operations
    tf.lite.OpsSet.SELECT_TF_OPS     # Enable TensorFlow operations
]

# Disable the experimental flag for lowering tensor list operations
converter._experimental_lower_tensor_list_ops = False

# Convert the model
tflite_model = converter.convert()

# Save the model to a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
